In [ ]:
val path = java.nio.file.FileSystems.getDefault().getPath("/Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/target/scala-2.11/histogrammodel_2.11-0.1.jar")
val x = ammonite.ops.Path(path)
interp.load.cp(x)

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram.Implicits._
import ru.hse.se.ba.danilin.paul.histogram.IHistogram

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

plotly.JupyterScala.init()

In [ ]:
implicit class DrawableHistogram[E](hist: IHistogram[E]) {
    def draw() = {
        val xs = hist.elementsPresent.toSeq
        val ys = xs.map(e => hist(e).toInt)
        Bar(xs.map(_.toString), ys).plot()
    }
}

In [ ]:
val v = "a a a b".toHistogram

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram.Histogram
val Histogram(h) = v

In [ ]:
val (x, y) = h.mapValues(_.toInt).unzip

Bar(x.toSeq, y.toSeq).plot()